### Import library
Nous allons importer les bibliothèque numpy et pandas qui sont necessaire pour le traitement.

In [47]:
import pandas as pd
import numpy as np

### Import Data
Nous allons importer les données depuis le répertoire RAwData pour enfin les traiter et transformer

In [48]:
n=['TimesTamp', 'ECP', 'Antenna', 'RSSI', 'Channel', 'Adress']
p='../1-RawData/20200122/Asc-p27-static-mobilFp-mobileP-V2__01-22-2020_16h_53m_28s.csv'
d=','
def importData(path, delimit,cols):
    return pd.read_csv(path, sep=delimit,names=cols)

data=importData(p,d,n)
data = data.drop([0],axis=0) #Suppression de la 1ere ligne

data.head(10)

,TimesTamp,ECP,Antenna,RSSI,Channel,Adress
1,1579699702897021,300000000000000000017882,2,-55.5,865.7,192.168.0.10
2,1579699702919896,300000000000000000017882,2,-56,865.7,192.168.0.10
3,1579699702951407,30000000000000000001785C,3,-54.5,865.7,192.168.0.10
4,1579699702957564,245390003868,3,-61.5,865.7,192.168.0.10
5,1579699702975300,245390003869,3,-61.5,865.7,192.168.0.10
6,1579699702993183,30000000000000000001785C,3,-54.5,865.7,192.168.0.10
7,1579699702994771,245390003868,3,-62,865.7,192.168.0.10
8,1579699703020167,245390003869,3,-61.5,865.7,192.168.0.10
9,1579699703039548,300000000000000000017857,4,-57.5,865.7,192.168.0.10
10,1579699703041112,300000000000000000017882,4,-58,865.7,192.168.0.10


### TYPAGE DES CHAMPS
Nos allons typer les différentes colonnes du dataSet avec les types primaires de python 

In [49]:
def typage(data):
    data['ECP']=data['ECP'].astype(str)
    data['TimesTamp']=data['TimesTamp'].astype('int64')
    data['RSSI']=data['RSSI'].astype('float64')
    data['Antenna']=data['Antenna'].astype('int64')
    return data

data=typage(data)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2162 entries, 1 to 2162
Data columns (total 6 columns):
TimesTamp    2162 non-null int64
ECP          2162 non-null object
Antenna      2162 non-null int64
RSSI         2162 non-null float64
Channel      2162 non-null object
Adress       2162 non-null object
dtypes: float64(1), int64(2), object(3)
memory usage: 118.2+ KB


### False Positif
cette section est exécuter que lors de la génération des données de d'apprentissage et de test. Connaissant les données fausses positives, nous allons définir une colonne pour toutes les données qui sera 1 pour les parasites et 1 pour les autres. 

In [51]:
#La liste de nos étiquettes falsePositive
fp_list=['245390003868','245390003869','300000000000000000017882','300000000000000000017884','300000000000000000017851',
         '300000000000000000017856','300000000000000000017857','300000000000000000017858','300000000000000000017860',
         '30000000000000000001785A','30000000000000000001785C','3000000000000000000178BC',
         '30000000000000000001785D','30000000000000000001785F']

def setFalsePositive(fp_list, dataSet):
    fp={}
    for e in dataSet.ECP:
        fp[e]=0
        for f in fp_list:
            if(str(e).strip('[]')==f):
                fp[e]=1
    return fp

### Génération du DataSet
Nous allons générer l'ensemble des colonnes necessaires pour l'apprentissage et la classification

In [52]:
from scipy.stats import variation 

def generateDataSet(data, groupedBy):
    grouped_df = data.groupby(groupedBy) #On groupe les données par ..."Etiquetes"
    
    dataSet = pd.DataFrame({}) #Création d'une nouvelle dataFrame
    
    #Création des variables intermédiaire pour la récupération des champs
    ecp=pd.Series(grouped_df.ECP.unique().index,name="ECP")
    rc= pd.Series(grouped_df.ECP.count(),name="readcount")
    minRSSI= pd.Series(grouped_df.RSSI.min(),name="minRssi")
    maxRSSI= pd.Series(grouped_df.RSSI.max(),name="maxRssi")
    mean=pd.Series(grouped_df.RSSI.mean(),name="meanRssi")
    startTime=pd.Series(grouped_df.TimesTamp.min(),name="startTime")
    endTime=pd.Series(grouped_df.TimesTamp.max(),name="endTime")
    diff=pd.Series(grouped_df.RSSI.diff,name="difference")
    
    #Mise à jour des champs de la dataFrame
    dataSet['ECP']=ecp.values
    dataSet['RC']=rc.values
    dataSet['MIN']=minRSSI.values
    dataSet['MAX']=maxRSSI.values
    dataSet['MEAN']=mean.values
    dataSet['START']=startTime.values
    dataSet['END']=endTime.values
    dataSet['DURATION']=endTime.values-startTime.values
    cv={}
    a1={}
    a2={}
    a3={}
    a4={}
    for key, item in grouped_df:
        cv[key]=variation(grouped_df['RSSI'].get_group(key).values,axis=0)
        a1[key]=0
        a2[key]=0
        a3[key]=0
        a4[key]=0
        for a in grouped_df['Antenna'].get_group(key).values:
            if(a==1): a1[key]+=1
            if(a==2): a2[key]+=1
            if(a==3): a3[key]+=1
            if(a==4): a4[key]+=1
                
    dataSet['CV']=pd.Series(cv).values
    dataSet['A1']=pd.Series(a1).values
    dataSet['A2']=pd.Series(a2).values
    dataSet['A3']=pd.Series(a3).values
    dataSet['A4']=pd.Series(a4).values
    
    #A commenter lors du 
    fp=setFalsePositive(fp_list, dataSet)  
    dataSet['FP']=pd.Series(fp).values  
    return dataSet

dataSet=generateDataSet(data,'ECP')
dataSet.head(10)

,ECP,RC,MIN,MAX,MEAN,START,END,DURATION,CV,A1,A2,A3,A4,FP
0,245390003868,210,-67.0,-58.5,-61.957143,1579699702957564,1579699747867390,44909826,-0.020082,7,5,198,0,1
1,245390003869,174,-63.0,-56.5,-60.583333,1579699702975300,1579699745158768,42183468,-0.016437,0,0,173,1,1
2,300000000000000000017856,1,-63.5,-63.5,-63.500000,1579699733327556,1579699733327556,0,-0.000000,0,1,0,0,1
3,300000000000000000017857,304,-63.5,-56.0,-57.626645,1579699703039548,1579699747950958,44911410,-0.015811,0,2,1,301,1
4,300000000000000000017858,1,-57.5,-57.5,-57.500000,1579699717786689,1579699717786689,0,-0.000000,0,0,1,0,1
5,30000000000000000001785C,287,-58.5,-53.0,-54.932056,1579699702951407,1579699745675824,42724417,-0.017785,3,0,284,0,1
6,30000000000000000001785D,309,-64.5,-58.0,-61.085761,1579699703042235,1579699748151155,45108920,-0.014276,2,11,0,296,1
7,30000000000000000001785E,4,-60.0,-58.5,-59.500000,1579699713693963,1579699714336538,642575,-0.010292,4,0,0,0,0
8,300000000000000000017862,11,-60.5,-54.0,-57.000000,1579699710512550,1579699715069601,4557051,-0.036071,5,0,6,0,0
9,300000000000000000017863,18,-60.5,-52.5,-57.305556,1579699731195273,1579699733021167,1825894,-0.037713,0,10,0,8,0


### Export Data
Nous allons sauvegarder les donées dans un fichier .csv

In [53]:
import time
def exportData(data, path):
    data.to_csv(path, index = None, header=False)
    

In [ ]:
tm = time.strftime("%Y-%b-%d-%H_%M_%S")
path='../2-TransformedData/export_'+tm+'.csv'
exportData(dataSet, path)

### Split Data
Nous allons découper les données en deux, 80% des trainSets et 20% des tesSets

In [64]:
def splitDataLE20(data):
    dataSet_np = np.random.rand(len(data)) <0.8
    trainSet=data[dataSet_np]
    testSet=data[~dataSet_np]
    return trainSet, testSet

from sklearn.model_selection import train_test_split
def sliptDataE20(data):
    X = data.iloc[:, :-1].values
    y = data['FP'].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
    trainSet=pd.DataFrame(X_train)
    trainSet[13]=y_train
    
    testSet = pd.DataFrame(X_test)
    testSet[13] = y_test
    return trainSet, testSet

In [66]:
tm = time.strftime("%Y-%b-%d-%H_%M_%S")
trainSet, testSet = sliptDataE20(dataSet)
#trainSet, testSet = splitDataLE20(dataSet)

pathTrain='../3-DataSet/trainSet_'+tm+'.csv'
exportData(dataSet, pathTrain)

pathTest='../3-DataSet/testSet_'+tm+'.csv'
exportData(dataSet, pathTest)